In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np
import pandas as pd

import torch
import torch.nn as nn

CMAP = colormaps.get_cmap("tab10")

# Overparameterized Exponent Data

In [ ]:
DATA_DIRECTORY = "overparameterizedExponent"
metadatadf: pd.DataFrame = pd.read_pickle(f"{DATA_DIRECTORY}/metadata.pkl")
display(metadatadf.columns)

In [ ]:
columns = ["taskName", "exponentCount", "timeSeries", "lossSeries", "coefficientDifferenceSeries"]
aggregatedData = []

targetData = metadatadf[
    (metadatadf["useAdjoint"] == True) &
    (metadatadf["dataSteps"] == 1000) &
    (metadatadf["batchSize"] == 20) &
    (metadatadf["batchTime"] == 10) &
    (metadatadf["numEpochs"] == 2000) &
    (metadatadf["hiddenLayerSize"] == 50) 
]

for _, row in targetData.iterrows():
    taskName = row["taskName"]
    exponentCount = row["exponentCount"]
    trueCoefficientMatrix: torch.Tensor = row["trueCoefficientMatrix"]

    rowDataFile = row["fileTimestamp"] + ".pkl"
    rowData: pd.DataFrame = pd.read_pickle(f"{DATA_DIRECTORY}/{rowDataFile}")

    timeSeries: pd.Series = rowData["time"].diff()
    lossSeries: pd.Series = rowData["loss"].apply(lambda x: x.item())
    coefficientDifferenceSeries: pd.Series = rowData["predCoefficientMatrix"].apply(lambda x: torch.mean(torch.square(trueCoefficientMatrix - x)).item())

    aggregatedData.append([
        taskName,
        exponentCount,
        timeSeries.to_list(),
        lossSeries.to_list(),
        coefficientDifferenceSeries.to_list(),
    ])
epochs = rowData["epoch"].to_list()

aggregateddf = pd.DataFrame(aggregatedData, columns=columns)
del aggregatedData
display(aggregateddf.groupby(["taskName", "exponentCount"]).aggregate("count").reset_index())

In [ ]:
timeSeriesdf = aggregateddf.groupby(["taskName", "exponentCount"])["timeSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
timeSeriesdf["mean"] = timeSeriesdf["timeSeries"].apply(lambda x: x.mean(axis=0))
timeSeriesdf["std"] = timeSeriesdf["timeSeries"].apply(lambda x: x.std(axis=0))

for taskName in timeSeriesdf["taskName"].unique():
    targetdf = timeSeriesdf[timeSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        exponentCount = row["exponentCount"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        epochDiff = np.diff(epochs, prepend=[0])
        meanPerEpoch = mean / (epochDiff  * 1e9)
        plt.plot(epochs, meanPerEpoch, label=exponentCount, color=CMAP(rowIndex))
        # plt.errorbar(epochs, mean, yerr=std, label=exponentCount)
    fig.legend(title="Exponent Count", ncols=2, loc="center", bbox_to_anchor=(0.8,0.2))
    plt.title(f"Epoch Time\nTask: {taskName}")
    plt.ylabel("Time (s)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  =[]
    barErrors  = []
    for _, row in targetdf.iterrows():
        exponentCount = row["exponentCount"]
        mean: np.ndarray = row["mean"]
        epochDiff = np.diff(epochs, prepend=[0])
        meanPerEpoch = mean / (epochDiff  * 1e9)
        error = np.nanstd(meanPerEpoch)
        meanPerEpoch = np.nanmean(meanPerEpoch)
        barLabels.append(exponentCount)
        barHeights.append(meanPerEpoch)
        barErrors.append(error)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=exponentCount, color=colors)
    plt.title(f"Average Epoch Time Over All Training\nTask: {taskName}")
    plt.ylabel("Time (s)")
    plt.xlabel("Exponent Count")
    plt.show()

In [ ]:
lossSeriesdf = aggregateddf.groupby(["taskName", "exponentCount"])["lossSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
lossSeriesdf["mean"] = lossSeriesdf["lossSeries"].apply(lambda x: x.mean(axis=0))
lossSeriesdf["std"] = lossSeriesdf["lossSeries"].apply(lambda x: x.std(axis=0))

for taskName in lossSeriesdf["taskName"].unique():
    targetdf = lossSeriesdf[lossSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        exponentCount = row["exponentCount"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        plt.plot(epochs, mean, label=exponentCount, color=CMAP(rowIndex))
        # plt.errorbar(epochs, mean, yerr=std, label=exponentCount)
    fig.legend(title="Exponent Count", ncols=2, loc="center", bbox_to_anchor=(0.8,0.8))
    plt.title(f"Loss\nTask: {taskName}")
    plt.ylabel("Loss (MAE)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  = []
    barErrors  = []
    for _, row in targetdf.iterrows():
        exponentCount = row["exponentCount"]
        mean = row["mean"][-1]
        std = row["std"][-1]
        barLabels.append(exponentCount)
        barHeights.append(mean)
        barErrors.append(std)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=exponentCount, color=colors)
    plt.title(f"Average Final Loss Over All Training\nTask: {taskName}")
    plt.ylabel("Loss (MAE)")
    plt.xlabel("Exponent Count")
    plt.show()

In [ ]:
coefficientDifferenceSeriesdf = aggregateddf.groupby(["taskName", "exponentCount"])["coefficientDifferenceSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
coefficientDifferenceSeriesdf["mean"] = coefficientDifferenceSeriesdf["coefficientDifferenceSeries"].apply(lambda x: x.mean(axis=0))
coefficientDifferenceSeriesdf["std"] = coefficientDifferenceSeriesdf["coefficientDifferenceSeries"].apply(lambda x: x.std(axis=0))

for taskName in coefficientDifferenceSeriesdf["taskName"].unique():
    targetdf = coefficientDifferenceSeriesdf[coefficientDifferenceSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        exponentCount = row["exponentCount"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        plt.plot(epochs, mean, label=exponentCount, color=CMAP(rowIndex))
        # plt.errorbar(epochs, mean, yerr=std, label=exponentCount)
    fig.legend(title="Exponent Count", ncols=2, loc="center", bbox_to_anchor=(0.8,0.8))
    plt.title(f"Coefficient Matrix Difference (MSE)\nTask: {taskName}")
    plt.ylabel("Error (MSE)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  = []
    barErrors  = []
    for _, row in targetdf.iterrows():
        exponentCount = row["exponentCount"]
        mean = row["mean"][-1]
        std = row["std"][-1]
        barLabels.append(exponentCount)
        barHeights.append(mean)
        barErrors.append(std)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=exponentCount, color=colors)
    plt.title(f"Final Coefficient Matrix Difference (MSE)\nTask: {taskName}")
    plt.ylabel("Error (MSE)")
    plt.xlabel("Exponent Count")
    plt.show()

# Optimizer Data

In [ ]:
DATA_DIRECTORY = "optimizer"
metadatadf: pd.DataFrame = pd.read_pickle(f"{DATA_DIRECTORY}/metadata.pkl")
display(metadatadf.columns)

In [ ]:
columns = ["taskName", "optimizer", "timeSeries", "lossSeries", "coefficientDifferenceSeries"]
aggregatedData = []

targetData = metadatadf[
    (metadatadf["useAdjoint"] == True) &
    (metadatadf["dataSteps"] == 1000) &
    (metadatadf["batchSize"] == 20) &
    (metadatadf["batchTime"] == 10) &
    (metadatadf["numEpochs"] == 2000) &
    (metadatadf["hiddenLayerSize"] == 50) 
]

for _, row in targetData.iterrows():
    taskName = row["taskName"]
    optimizer = row["optimizer"]
    trueCoefficientMatrix: torch.Tensor = row["trueCoefficientMatrix"]

    rowDataFile = row["fileTimestamp"] + ".pkl"
    rowData: pd.DataFrame = pd.read_pickle(f"{DATA_DIRECTORY}/{rowDataFile}")

    timeSeries: pd.Series = rowData["time"].diff()
    lossSeries: pd.Series = rowData["loss"].apply(lambda x: x.item())
    coefficientDifferenceSeries: pd.Series = rowData["predCoefficientMatrix"].apply(lambda x: torch.mean(torch.square(trueCoefficientMatrix - x)).item())

    aggregatedData.append([
        taskName,
        optimizer,
        timeSeries.to_list(),
        lossSeries.to_list(),
        coefficientDifferenceSeries.to_list(),
    ])
epochs = rowData["epoch"].to_list()

aggregateddf = pd.DataFrame(aggregatedData, columns=columns)
del aggregatedData
display(aggregateddf.groupby(["taskName", "optimizer"]).aggregate("count").reset_index())

In [ ]:
timeSeriesdf = aggregateddf.groupby(["taskName", "optimizer"])["timeSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
timeSeriesdf["mean"] = timeSeriesdf["timeSeries"].apply(lambda x: x.mean(axis=0))
timeSeriesdf["std"] = timeSeriesdf["timeSeries"].apply(lambda x: x.std(axis=0))

for taskName in timeSeriesdf["taskName"].unique():
    targetdf = timeSeriesdf[timeSeriesdf["taskName"] == taskName]
    
    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        optimizer = row["optimizer"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        epochDiff = np.diff(epochs, prepend=[0])
        meanPerEpoch = mean / (epochDiff  * 1e9)
        plt.plot(epochs, meanPerEpoch, label=optimizer, color=CMAP(rowIndex))
    fig.legend(title="Optimizer", ncols=4, loc="center", bbox_to_anchor=(0.5,-0.0))
    plt.title(f"Epoch Time\nTask: {taskName}")
    plt.ylabel("Time (s)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  =[]
    barErrors  = []
    for _, row in targetdf.iterrows():
        optimizer = row["optimizer"]
        mean: np.ndarray = row["mean"]
        epochDiff = np.diff(epochs, prepend=[0])
        meanPerEpoch = mean / (epochDiff  * 1e9)
        error = np.nanstd(meanPerEpoch)
        meanPerEpoch = np.nanmean(meanPerEpoch)
        barLabels.append(optimizer)
        barHeights.append(meanPerEpoch)
        barErrors.append(error)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=optimizer, color=colors)
    plt.title(f"Average Epoch Time Over All Training\nTask: {taskName}")
    plt.ylabel("Time (s)")
    plt.xlabel("Optimizer")
    plt.show()

In [ ]:
lossSeriesdf = aggregateddf.groupby(["taskName", "optimizer"])["lossSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
lossSeriesdf["mean"] = lossSeriesdf["lossSeries"].apply(lambda x: x.mean(axis=0))
# lossSeriesdf["mean"] = lossSeriesdf["mean"].apply(lambda x: x / x[0])
lossSeriesdf["std"] = lossSeriesdf["lossSeries"].apply(lambda x: x.std(axis=0))

for taskName in lossSeriesdf["taskName"].unique():
    targetdf = lossSeriesdf[lossSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        optimizer = row["optimizer"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        plt.plot(epochs, mean, label=optimizer, color=CMAP(rowIndex))
    fig.legend(title="Optimizer", ncols=4, loc="center", bbox_to_anchor=(0.5,-0.0))
    plt.title(f"Loss\nTask: {taskName}")
    plt.ylabel("Loss (MAE)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  = []
    barErrors  = []
    for _, row in targetdf.iterrows():
        optimizer = row["optimizer"]
        mean = row["mean"][-1]
        std = row["std"][-1]
        barLabels.append(optimizer)
        barHeights.append(mean)
        barErrors.append(std)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=optimizer, color=colors)
    plt.title(f"Average Final Loss Over All Training\nTask: {taskName}")
    plt.ylabel("Loss (MAE)")
    plt.xlabel("Optimizer")
    plt.show()

In [ ]:
coefficientDifferenceSeriesdf = aggregateddf.groupby(["taskName", "optimizer"])["coefficientDifferenceSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
coefficientDifferenceSeriesdf["mean"] = coefficientDifferenceSeriesdf["coefficientDifferenceSeries"].apply(lambda x: x.mean(axis=0))
coefficientDifferenceSeriesdf["std"] = coefficientDifferenceSeriesdf["coefficientDifferenceSeries"].apply(lambda x: x.std(axis=0))

for taskName in coefficientDifferenceSeriesdf["taskName"].unique():
    targetdf = coefficientDifferenceSeriesdf[coefficientDifferenceSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        optimizer = row["optimizer"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        plt.plot(epochs, mean, label=optimizer, color=CMAP(rowIndex))
        # plt.errorbar(epochs, mean, yerr=std, label=optimizer)
    fig.legend(title="Optimizer", ncols=4, loc="center", bbox_to_anchor=(0.5,-0.0))
    plt.title(f"Coefficient Matrix Difference (MSE)\nTask: {taskName}")
    plt.ylabel("Error (MSE)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  = []
    barErrors  = []
    for _, row in targetdf.iterrows():
        optimizer = row["optimizer"]
        mean = row["mean"][-1]
        std = row["std"][-1]
        barLabels.append(optimizer)
        barHeights.append(mean)
        barErrors.append(std)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=optimizer, color=colors)
    plt.title(f"Final Coefficient Matrix Difference (MSE)\nTask: {taskName}")
    plt.ylabel("Error (MSE)")
    plt.xlabel("Optimizer")
    plt.show()

# Loss Data

In [ ]:
DATA_DIRECTORY = "loss"
metadatadf: pd.DataFrame = pd.read_pickle(f"{DATA_DIRECTORY}/metadata.pkl")
display(metadatadf.columns)

In [ ]:
columns = ["taskName", "lossName", "timeSeries", "lossSeries", "coefficientDifferenceSeries"]
aggregatedData = []

targetData = metadatadf[
    (metadatadf["useAdjoint"] == True) &
    (metadatadf["dataSteps"] == 1000) &
    (metadatadf["batchSize"] == 20) &
    (metadatadf["batchTime"] == 10) &
    (metadatadf["numEpochs"] == 2000) &
    (metadatadf["hiddenLayerSize"] == 50) 
]

for _, row in targetData.iterrows():
    taskName = row["taskName"]
    lossName = row["loss"]
    trueCoefficientMatrix: torch.Tensor = row["trueCoefficientMatrix"]

    rowDataFile = row["fileTimestamp"] + ".pkl"
    rowData: pd.DataFrame = pd.read_pickle(f"{DATA_DIRECTORY}/{rowDataFile}")

    timeSeries: pd.Series = rowData["time"].diff()
    lossSeries: pd.Series = rowData["loss"].apply(lambda x: x.item())
    coefficientDifferenceSeries: pd.Series = rowData["predCoefficientMatrix"].apply(lambda x: torch.mean(torch.square(trueCoefficientMatrix - x)).item())

    aggregatedData.append([
        taskName,
        lossName,
        timeSeries.to_list(),
        lossSeries.to_list(),
        coefficientDifferenceSeries.to_list(),
    ])
epochs = rowData["epoch"].to_list()

aggregateddf = pd.DataFrame(aggregatedData, columns=columns)
del aggregatedData
display(aggregateddf.groupby(["taskName", "lossName"]).aggregate("count").reset_index())

In [ ]:
timeSeriesdf = aggregateddf.groupby(["taskName", "lossName"])["timeSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
timeSeriesdf["mean"] = timeSeriesdf["timeSeries"].apply(lambda x: x.mean(axis=0))
timeSeriesdf["std"] = timeSeriesdf["timeSeries"].apply(lambda x: x.std(axis=0))

for taskName in timeSeriesdf["taskName"].unique():
    targetdf = timeSeriesdf[timeSeriesdf["taskName"] == taskName]
    
    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        lossName = row["lossName"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        epochDiff = np.diff(epochs, prepend=[0])
        meanPerEpoch = mean / (epochDiff  * 1e9)
        plt.plot(epochs, meanPerEpoch, label=lossName, color=CMAP(rowIndex))
    fig.legend(title="Loss Function", ncols=4, loc="center", bbox_to_anchor=(0.5,-0.0))
    plt.title(f"Epoch Time\nTask: {taskName}")
    plt.ylabel("Time (s)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  =[]
    barErrors  = []
    for _, row in targetdf.iterrows():
        lossName = row["lossName"]
        mean: np.ndarray = row["mean"]
        epochDiff = np.diff(epochs, prepend=[0])
        meanPerEpoch = mean / (epochDiff  * 1e9)
        error = np.nanstd(meanPerEpoch)
        meanPerEpoch = np.nanmean(meanPerEpoch)
        barLabels.append(lossName)
        barHeights.append(meanPerEpoch)
        barErrors.append(error)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=lossName, color=colors)
    plt.title(f"Average Epoch Time Over All Training\nTask: {taskName}")
    plt.ylabel("Time (s)")
    plt.xlabel("Loss Function")
    plt.show()

In [ ]:
lossSeriesdf = aggregateddf.groupby(["taskName", "lossName"])["lossSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
lossSeriesdf["mean"] = lossSeriesdf["lossSeries"].apply(lambda x: x.mean(axis=0))
# lossSeriesdf["mean"] = lossSeriesdf["mean"].apply(lambda x: x / x[0])
lossSeriesdf["std"] = lossSeriesdf["lossSeries"].apply(lambda x: x.std(axis=0))

for taskName in lossSeriesdf["taskName"].unique():
    targetdf = lossSeriesdf[lossSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        lossName = row["lossName"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        plt.plot(epochs, mean, label=lossName, color=CMAP(rowIndex))
    fig.legend(title="Loss Function", ncols=4, loc="center", bbox_to_anchor=(0.5,-0.0))
    plt.title(f"Loss by Loss Function\nTask: {taskName}")
    plt.ylabel("Loss (relative)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  = []
    barErrors  = []
    for _, row in targetdf.iterrows():
        lossName = row["lossName"]
        mean = row["mean"][-1]
        std = row["std"][-1]
        barLabels.append(lossName)
        barHeights.append(mean)
        barErrors.append(std)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=lossName, color=colors)
    plt.title(f"Average Final Loss Over All Training\nTask: {taskName}")
    plt.ylabel("Loss (relative)")
    plt.xlabel("Loss Function")
    plt.show()

In [ ]:
coefficientDifferenceSeriesdf = aggregateddf.groupby(["taskName", "lossName"])["coefficientDifferenceSeries"] \
    .apply(lambda x: np.vstack(x)) \
    .reset_index()
coefficientDifferenceSeriesdf["mean"] = coefficientDifferenceSeriesdf["coefficientDifferenceSeries"].apply(lambda x: x.mean(axis=0))
coefficientDifferenceSeriesdf["std"] = coefficientDifferenceSeriesdf["coefficientDifferenceSeries"].apply(lambda x: x.std(axis=0))

for taskName in coefficientDifferenceSeriesdf["taskName"].unique():
    targetdf = coefficientDifferenceSeriesdf[coefficientDifferenceSeriesdf["taskName"] == taskName]

    fig = plt.figure(figsize=(8,8))
    for rowIndex, (_, row) in enumerate(targetdf.iterrows()):
        rowIndex = int(rowIndex) # pyright: ignore[reportArgumentType]
        lossName = row["lossName"]
        mean: np.ndarray = row["mean"]
        std: np.ndarray = row["std"]

        plt.plot(epochs, mean, label=lossName, color=CMAP(rowIndex))
        # plt.errorbar(epochs, mean, yerr=std, label=lossName)
    fig.legend(title="Loss Function", ncols=4, loc="center", bbox_to_anchor=(0.5,-0.0))
    plt.title(f"Coefficient Matrix Difference (MSE)\nTask: {taskName}")
    plt.ylabel("Error (MSE)")
    plt.xlabel("Epoch")
    plt.show()

    fig = plt.figure(figsize=(8,8))
    barLabels = []
    barHeights  = []
    barErrors  = []
    for _, row in targetdf.iterrows():
        lossName = row["lossName"]
        mean = row["mean"][-1]
        std = row["std"][-1]
        barLabels.append(lossName)
        barHeights.append(mean)
        barErrors.append(std)
    colors = [CMAP(i) for i in range(len(barLabels))]
    plt.bar(barLabels, barHeights, yerr=barErrors, label=lossName, color=colors)
    plt.title(f"Final Coefficient Matrix Difference (MSE)\nTask: {taskName}")
    plt.ylabel("Error (MSE)")
    plt.xlabel("Loss Function")
    plt.show()